# Assessing trade quality of past trades in hindsight
### Main Objectives:

Backtesting i.e., See if trading philosophy is worth digitizing 

Strategy optimization i.e., can this philosphy be easily improve



### Get past trades and rate timing against ema crossover

Ratings performed as    

In [17]:
# Imports
# Import necessary libraries
import numpy as np
import pandas as pd
import requests
import datetime
import hashlib
from difflib import get_close_matches
import hmac
import base64
import time
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import urllib.parse

# Import config file
import sys
sys.path.insert(0, '../')
import config as cfg

In [18]:
api_key = cfg.api_key
private_key = cfg.api_priv

In [19]:
api_url = "https://api.kraken.com"

# Create dictionary to map API endpoints to their respective names and respective request types
api_endpoints = {
    'Assets': '/0/public/Assets',
    'AssetPairs': '/0/public/AssetPairs',
    'Ticker': '/0/public/Ticker',
    'OHLC': '/0/public/OHLC',
    'default OHLC': '/0/public/OHLC',

    'Balance': '/0/private/Balance',
    'ExtendedBalance': '/0/private/BalanceEx',
    'Ledgers': '/0/private/Ledgers',
    'QueryLedgers': '/0/private/QueryLedgers',
    'TradeVolume': '/0/private/TradeVolume',
    'TradesHistory': '/0/private/TradesHistory',
    'OpenOrders': '/0/private/OpenOrders',

}

# Function to generate a nonce
def generate_nonce():
    nonce = str(int(1000 * time.time()))
    return nonce

# Function to get Kraken signature
def get_kraken_signature(urlpath, data, secret):
    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode() 

# Function to make Kraken API request
def kraken_request(uri_path, data, api_key, api_sec, headers=None):
    if headers is None:
        headers = {}

    headers['API-Key'] = api_key
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

#Function to make more non-trivial Kraken API get request
def kraken_get_request(uri_path, data=None, headers=None):
    if uri_path != api_endpoints['OHLC']:
        if headers is None:
            headers = {
                'Accept': 'application/json'
            }
        
        headers.update(headers)
        req = requests.get((api_url + uri_path), headers=headers, data=data)
    elif uri_path == api_endpoints['OHLC']:
        temp_endpoint = api_endpoints['OHLC'] + '?pair=' + data['pair'] + '&interval=' + data['interval']
        req = requests.get((api_url + temp_endpoint), headers=headers)
    return req


In [20]:
# Possible intervals: 1, 5, 15, 30, 60, 240, 1440, 10080, 21600 in minutes i.e., 1 minute, 5 minutes, 15 minutes, 30 minutes, 1 hour, 4 hours, 1 day, 1 week, 1 month
# Possible tenures: 1D (1440), 7D (10080), 1M (43200), 3M (129600), 6M (259200), 1Y (518400) - corresponding intervals are tenure/720 to maximize data points from a single request
possible_intervals =[1, 5, 15, 30, 60, 240, 1440, 10080, 21600]
possible_timeframes = {'1D': 1440, '7D': 10080, '1M': 43200, '3M': 129600, '6M': 259200, '1Y': 518400}

tenure = '7D'

In [27]:
# 1. Get Historical OHLC Data of the Asset(s) using Kraken REST API
# Function to grab the OHLC data for a given list of asset pairs
def grab_ohlc_data(assetPairs,tenure):
    # divide timerframe by 720 to get the interval but use the next larger closet possible interval
    interval = min([i for i in possible_intervals if i >= possible_timeframes[tenure]/720], default=possible_intervals[-1])
    interval = str(interval)
    # Construct since parameter for the OHLC request using tenure and datetime unix converted timestamp, i.e., subtracting the tenure from the current time and equating it to the since parameter
    since = int(time.time()) - possible_timeframes[tenure]*60
    since = str(since)

    # Construct the Kraken API request and get the OHLC data for the given asset pairs, ohlc grabbing requires use of a temporary endpoint for the OHLC url
    ohlcDict = {}
    for assetPair in assetPairs:
        resp = kraken_get_request(api_endpoints['OHLC'], {"pair": assetPair, "interval": interval, "since": since}).json()
        if resp['error'] == KeyError:
            #skip this asset pair
            continue
        ohlcDict[assetPair] = list(resp['result'].values())
        # To process the response, we need to extract the OHLC data from the response particularly the tick data array and the last timestamp
    # Append the OHLC data to a dataframe and return the dataframe with columns: Time, Open, High, Low, Close, Volume, Count, name it after the asset pair
    return ohlcDict

def grab_trades_history():
    # Get trade history
    resp = kraken_request(api_endpoints['TradesHistory'], {"trades": True}, api_key, private_key).json()

    # Extract the trade history data from the response
    tradeHistory = resp['result']

def grab_open_orders():
    #  resp = kraken_request(api_endpoints['ExtendedBalance'], {"nonce": generate_nonce(),}, api_key, api_sec).json()
    # Get open orders
    # Generate nonce
    # Define payload
    payload = {
        "nonce": generate_nonce(),
        "trades": True
    }
    
    # Make the API request
    resp = kraken_request(api_endpoints['OpenOrders'], payload, api_key, private_key)
    resp_json = resp.json()
    
    # Check for errors in the response
    if resp_json.get('error'):
        raise Exception(f"Kraken API Error: {resp_json['error']}")
    
    # Extract and return open orders data
    open_orders = resp_json['result']['open']
    return open_orders


In [28]:
assetList = ['XXRP','SOL', 'XXBT', 'TIA', 'TAO', 'OP','FLR','AXS']

ConnectionError: HTTPSConnectionPool(host='api.kraken.com', port=443): Max retries exceeded with url: /0/private/BalanceEx (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001C8A65FA850>: Failed to resolve 'api.kraken.com' ([Errno 11002] getaddrinfo failed)"))